In [ ]:
import os
datadir = '/group/jug/ashesh/data/Elisa3D'
config_fpath = '/group/jug/ashesh/training/disentangle/2408/D29-M3-S0-L0/17/'
config_fpath = config_fpath[0] + config_fpath[1:].strip('/')
pred_fpath = '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M5_Sk0/pred_training_disentangle_2408_D29-M3-S0-L0_17_1.tif'
assert os.path.basename(pred_fpath).replace('.tif', '').split('_')[-2] == os.path.basename(config_fpath), 'different model'

In [ ]:
from disentangle.config_utils import load_config
config = load_config(config_fpath)

In [ ]:
from disentangle.data_loader.train_val_data import get_train_val_data
from disentangle.core.data_split_type import DataSplitType

data = get_train_val_data(config.data,
                       datadir,
                       DataSplitType.Test,
                       val_fraction=config.training.val_fraction,
                       test_fraction=config.training.test_fraction) 
data.shape

In [ ]:
from disentangle.core.tiff_reader import  load_tiff
import json
def load_pred(fpath):
    pred = load_tiff(fpath)
    with open(fpath.replace('.tif', '.json'), 'r') as f:
        json_data = json.load(f)
    assert json_data['factor'] == '1.0', 'factor is not 1.0'
    return pred + float(json_data['offset'])

pred = load_pred(pred_fpath)
pred.shape

In [ ]:
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
res= 4
def plot_one_z(zidx):
    fig,ax = plt.subplots(figsize=(16,4),ncols=4,nrows=1)
    ax[0].imshow(data[0,zidx,::res,::res,0])
    ax[1].imshow(pred[0,zidx,::res,::res,0])

    ax[2].imshow(data[0,zidx,::res,::res,1])
    ax[3].imshow(pred[0,zidx,::res,::res,1])

_ = interact(plot_one_z, zidx=widgets.IntSlider(min=0,max=pred.shape[1]-1,step=1,value=1)) 



In [ ]:
pred.mean(), data.mean()

In [ ]:
from disentangle.scripts.evaluate import  compute_high_snr_stats

stats = [compute_high_snr_stats(config,data[:,z], pred[:,z], verbose=False) for z in range(data.shape[1])]

In [ ]:
import numpy as np
psnr_arr = np.array([stats[i]['rangeinvpsnr'] for i in range(len(stats))])[...,0]
psnr_arr.mean(axis=0), psnr_arr.std(axis=0)

In [ ]:
from disentangle.core.tiff_reader import load_tiff
load_tiff('/group/jug/ashesh/data/Elisa/N2V_Elisa/n2v_input.tif').shape

In [ ]:
! ls /group/jug/ashesh/data/Elisa3D/20240725/

In [ ]:
break here

## N2V denoising data creation

In [ ]:
from disentangle.data_loader.train_val_data import get_train_val_data
from disentangle.core.data_split_type import DataSplitType

data = get_train_val_data(config.data,
                       datadir,
                       DataSplitType.Train,
                       val_fraction=config.training.val_fraction,
                       test_fraction=config.training.test_fraction) 
data.shape

In [ ]:
import numpy as np

n_list = []
z_list = []
for i in range(len(data)):
    z_idx = np.random.choice(range(data.shape[1]), size=3, replace=False)
    n_list.append([i])
    z_list.append(z_idx.tolist())


In [ ]:
reduced_data = data[n_list,z_list]
reduced_data = reduced_data.reshape(-1, reduced_data.shape[2], reduced_data.shape[3], reduced_data.shape[4])
reduced_data.shape

In [ ]:
plt.imshow(reduced_data[16,...,1])

In [ ]:
from disentangle.core.tiff_reader import save_tiff
save_tiff('/group/jug/ashesh/data/Elisa3D/20240725/n2v_denoising/raw_ch0.tif', reduced_data[...,0])
save_tiff('/group/jug/ashesh/data/Elisa3D/20240725/n2v_denoising/raw_ch1.tif', reduced_data[...,1], )

In [ ]:
from disentangle.core.tiff_reader import load_tiff
data = load_tiff('/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk0/pred_training_disentangle_2408_D12-M3-S0-L8_3_1.tif')

In [ ]:
import matplotlib.pyplot as plt
_,ax = plt.subplots(figsize=(8,4),ncols=2)
idx = 5
ax[0].imshow(data[0,0,idx,...,0])
ax[1].imshow(data[0,0,idx,...,1])